<a href="https://colab.research.google.com/github/josvaldes/trabajoGradoMCD/blob/SegundaBD/scrapingOoklaFinal4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# -*- coding: utf-8 -*-
"""
Notebook N.º 2 – Construcción incremental de base Ookla (Speedtest)
Versión v4 – Octubre 2025
Autor: José Eduardo Valdés Castro
Proyecto: Validación cruzada de información sobre conectividad en Colombia

Objetivo:
Automatizar la descarga, decodificación y persistencia incremental de los datos públicos
de velocidad de Internet (Ookla Open Data) para aislar los registros pertenecientes a Colombia
mediante decodificación de tiles (quadkeys) y cruce geográfico con municipios y departamentos.
"""

# ============================================================
# === 1. Preparar entorno y montar Google Drive ==============
# ============================================================

try:
    import geopandas, mercantile, duckdb, shapely
except ImportError:
    !pip install geopandas mercantile shapely duckdb tqdm requests bs4 pyproj --quiet


from google.colab import drive
import os, requests, pandas as pd, geopandas as gpd, duckdb
from shapely.geometry import Point
from tqdm import tqdm
import mercantile

# Montar Drive
drive.mount('/content/gdrive')

# Definir rutas principales
base_dir = "/content/gdrive/MyDrive/trabajoGrado"
ruta_ookla = os.path.join(base_dir, "ookla_colombia")          # Carpeta con todos los parquets globales
ruta_datos = os.path.join(base_dir, "ookla_datos")             # Carpeta para base persistente y logs
os.makedirs(ruta_ookla, exist_ok=True)
os.makedirs(ruta_datos, exist_ok=True)

db_path  = os.path.join(ruta_datos, "ookla_colombia.duckdb")
log_path = os.path.join(ruta_datos, "registro_procesados.csv")

print(f"✅ Entorno preparado:\n📂 Fuente: {ruta_ookla}\n💾 Base: {db_path}")

# ============================================================
# === 2. Descarga directa desde el bucket Ookla (S3 CLI) =====
# ============================================================

import os

# Asegurar que el AWS CLI está instalado
!apt-get -qq install awscli > /dev/null

base_dir = "/content/gdrive/MyDrive/trabajoGrado/ookla_colombia/"
years = range(2019, 2025)
quarters = [1, 2, 3, 4]

for year in years:
    for q in quarters:
        local_path = f"{base_dir}{year}/quarter={q}/"
        os.makedirs(local_path, exist_ok=True)

        # Verificar si ya hay archivos .parquet
        if any(fname.endswith(".parquet") for fname in os.listdir(local_path)):
            print(f"✅ Archivos ya existen en {year} Q{q}, se omite descarga.")
            continue

        print(f"⬇️ Descargando {year} Q{q}...")
        cmd = f"aws s3 cp --no-sign-request --recursive s3://ookla-open-data/parquet/performance/type=fixed/year={year}/quarter={q}/ {local_path}"
        os.system(cmd)

print("\n🎯 Descarga completada. Archivos guardados en Drive.")

# ============================================================
# === 3. Función para decodificar quadkeys → lat/lon =========
# ============================================================

def decodificar_quadkeys(df):
    """
    Decodifica la columna 'quadkey' o 'tile' y genera columnas 'lat' y 'lon'
    representando el centro del tile correspondiente.
    """
    col_ref = "quadkey" if "quadkey" in df.columns else "tile"
    lats, lons = [], []

    for qk in tqdm(df[col_ref], desc="Decodificando quadkeys", leave=False):
        try:
            tile = mercantile.quadkey_to_tile(str(qk))
            bbox = mercantile.bounds(tile)
            lon = (bbox.west + bbox.east) / 2
            lat = (bbox.north + bbox.south) / 2
            lats.append(lat)
            lons.append(lon)
        except Exception:
            lats.append(None)
            lons.append(None)

    df["lat"] = lats
    df["lon"] = lons
    return df

# ============================================================
# === 4. Descarga y normalización de shapes (GeoBoundaries) ==
# ============================================================

import geopandas as gpd
import os, requests

# Carpeta donde se almacenarán los shapes
shape_dir = "/content/gdrive/MyDrive/trabajoGrado/shapes"
os.makedirs(shape_dir, exist_ok=True)

# Configuración de fuentes (GeoBoundaries API)
shapes = {
    "departamentos": {
        "file": os.path.join(shape_dir, "colombia_departamentos.geojson"),
        "meta_url": "https://www.geoboundaries.org/api/current/gbOpen/COL/ADM1/"
    },
    "municipios": {
        "file": os.path.join(shape_dir, "colombia_municipios.geojson"),
        "meta_url": "https://www.geoboundaries.org/api/current/gbOpen/COL/ADM2/"
    }
}

def descargar_shape(name, meta_url, shape_file, min_size=50000):
    """
    Descarga el GeoJSON si no existe o está dañado.
    Usa la API oficial de GeoBoundaries para garantizar disponibilidad.
    """
    if not os.path.exists(shape_file) or os.path.getsize(shape_file) < min_size:
        print(f"⚠️ Descargando {name}...")
        meta = requests.get(meta_url).json()
        geojson_url = meta["gjDownloadURL"]
        r = requests.get(geojson_url)
        r.raise_for_status()
        with open(shape_file, "wb") as f:
            f.write(r.content)
        print(f"✅ {name.capitalize()} descargado en {shape_file}")
    else:
        print(f"✔️ {name.capitalize()} ya existe en disco ({shape_file})")

    # Cargar y proyectar correctamente
    gdf = gpd.read_file(shape_file).to_crs("EPSG:4326")
    print(f"📊 {name.capitalize()} cargado con {len(gdf)} registros.")
    return gdf

# Descargar ambos niveles
colombia_departamentos = descargar_shape(
    "departamentos", shapes["departamentos"]["meta_url"], shapes["departamentos"]["file"]
)
colombia_municipios = descargar_shape(
    "municipios", shapes["municipios"]["meta_url"], shapes["municipios"]["file"]
)

# === Normalizar columnas ===
gdf_dep = colombia_departamentos.rename(columns={"shapeName": "departamento"})[["departamento", "geometry"]]
gdf_mun = colombia_municipios.rename(columns={"shapeName": "municipio"})[["municipio", "geometry"]]
gdf_dep = gdf_dep.set_crs("EPSG:4326")
gdf_mun = gdf_mun.set_crs("EPSG:4326")

print("\n✅ Shapes normalizados correctamente:")
print(f" - Departamentos: {len(gdf_dep)} registros")
print(f" - Municipios: {len(gdf_mun)} registros")


# ============================================================
# === 5. Función de procesamiento de archivos ================
# ============================================================

def procesar_archivo_ookla(fpath, gdf_mun, gdf_dep,
                           lon_min=-79.5, lat_min=-4.3, lon_max=-66.8, lat_max=13.5):
    """
    Procesa un archivo .parquet global: decodifica quadkeys, filtra por coordenadas
    de Colombia y cruza con municipios/departamentos normalizados.
    """
    df = pd.read_parquet(fpath)
    if "quadkey" not in df.columns and "tile" not in df.columns:
        raise ValueError("El archivo no contiene columna 'quadkey' o 'tile'.")

    df = decodificar_quadkeys(df)

    # Filtrar por bounding box de Colombia
    df = df[(df["lat"] >= lat_min) & (df["lat"] <= lat_max) &
            (df["lon"] >= lon_min) & (df["lon"] <= lon_max)]

    # Convertir a GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat), crs="EPSG:4326")

    # === Enriquecer con municipios (ADM2) ===
    gdf = gpd.sjoin(gdf, gdf_mun, predicate="within", how="left")
    gdf = gdf.rename(columns={"municipio": "municipio"}).drop(columns=[c for c in gdf.columns if c.endswith("_right")], errors="ignore")

    # === Enriquecer con departamentos (ADM1) ===
    gdf = gpd.sjoin(gdf, gdf_dep, predicate="within", how="left", rsuffix="_dep")
    gdf = gdf.rename(columns={"departamento": "departamento"}).drop(columns=[c for c in gdf.columns if c.endswith("_right")], errors="ignore")

    # Limpiar geometría
    gdf = gdf.drop(columns="geometry")
    return gdf


# ============================================================
# === 6. Cargar log y detectar archivos nuevos ===============
# ============================================================

from glob import glob

# Cargar log de procesados
if os.path.exists(log_path):
    log = pd.read_csv(log_path)
    procesados = set(log["archivo"])
else:
    log = pd.DataFrame(columns=["archivo", "registros", "fecha_procesado"])
    procesados = set()

# Buscar .parquet en TODAS las subcarpetas (por año y trimestre)
archivos = glob(os.path.join(ruta_ookla, "**/*.parquet"), recursive=True)
archivos = [f for f in archivos if os.path.isfile(f)]
nuevos = [f for f in archivos if os.path.basename(f) not in procesados]

print(f"\n📦 Archivos detectados en estructura: {len(archivos)}")
for a in archivos[:5]:
    print("   •", a.split("/MyDrive/")[-1])

print(f"🆕 Nuevos por procesar: {len(nuevos)}")


# ============================================================
# === 7. Crear o actualizar base de datos (versión segura) ===
# ============================================================

def crear_o_actualizar_base(nuevos):
    """
    Procesa e inserta de forma incremental los archivos Ookla,
    garantizando que no se dupliquen registros ya cargados.
    """
    print("\n🚀 Iniciando proceso de carga incremental...")

    # Cargar shapes actualizados
    gdf_mun, gdf_dep = colombia_municipios, colombia_departamentos
    con = duckdb.connect(db_path)
    total_insertados = 0

    for archivo in nuevos:
        fpath = os.path.join(ruta_ookla, archivo)

        try:
            print(f"\n🔄 Procesando {fpath}...")

            # ✅ Verificar si existe tabla base
            existe_tabla = con.execute("""
                SELECT COUNT(*) FROM information_schema.tables
                WHERE table_name = 'ookla_geo'
            """).fetchone()[0] > 0

            # ✅ Verificar si el archivo ya está insertado (según quadkeys)
            if existe_tabla:
                try:
                    existe_registro = con.execute(f"""
                        SELECT COUNT(*) FROM ookla_geo
                        WHERE quadkey IN (SELECT DISTINCT quadkey FROM read_parquet('{fpath}'))
                    """).fetchone()[0]
                except Exception:
                    existe_registro = 0

                if existe_registro > 0:
                    print(f"⏭️ {os.path.basename(fpath)} ya estaba cargado en la base. Se omite inserción.")
                    continue

            # ✅ Procesar archivo (decodificación, georreferenciación y cruce)
            gdf = procesar_archivo_ookla(fpath, gdf_mun, gdf_dep)

            # Crear tabla base si no existe
            if not existe_tabla:
                con.execute("CREATE TABLE ookla_geo AS SELECT * FROM gdf LIMIT 0")
                print("🧱 Tabla 'ookla_geo' creada en la base.")

            # Registrar DataFrame temporal e insertar
            con.register("temp", gdf)
            con.execute("INSERT INTO ookla_geo SELECT * FROM temp")
            total_insertados += len(gdf)

            # Registrar en el log
            log.loc[len(log)] = [fpath, len(gdf), pd.Timestamp.now()]
            print(f"✅ Insertados {len(gdf):,} registros desde {os.path.basename(fpath)}.")

        except Exception as e:
            print(f"⚠️ Error general en {os.path.basename(fpath)}: {e}")

    # Cerrar conexión y guardar log
    con.close()
    log.to_csv(log_path, index=False)

    print(f"\n🧾 Log actualizado con {len(nuevos)} archivos procesados.")
    print(f"📊 Total insertados en esta ejecución: {total_insertados:,} registros.")
    print("🎯 Base Ookla lista para análisis (Notebook 3 - INCTIC).")



# ============================================================
# === 8. Flujo principal =====================================
# ============================================================

if not os.path.exists(db_path):
    print("⚙️ No existe base persistente. Creando nueva base...")
    crear_o_actualizar_base(archivos)
elif nuevos:
    print("✅ Base existente detectada. Actualizando con nuevos trimestres...")
    crear_o_actualizar_base(nuevos)
else:
    print("🟢 Base actualizada. No hay nuevos archivos.")

print("\n🎯 Proceso completado. Base Ookla lista para análisis (Notebook 3 - INCTIC).")


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
✅ Entorno preparado:
📂 Fuente: /content/gdrive/MyDrive/trabajoGrado/ookla_colombia
💾 Base: /content/gdrive/MyDrive/trabajoGrado/ookla_datos/ookla_colombia.duckdb
✅ Archivos ya existen en 2019 Q1, se omite descarga.
✅ Archivos ya existen en 2019 Q2, se omite descarga.
✅ Archivos ya existen en 2019 Q3, se omite descarga.
✅ Archivos ya existen en 2019 Q4, se omite descarga.
✅ Archivos ya existen en 2020 Q1, se omite descarga.
✅ Archivos ya existen en 2020 Q2, se omite descarga.
✅ Archivos ya existen en 2020 Q3, se omite descarga.
✅ Archivos ya existen en 2020 Q4, se omite descarga.
✅ Archivos ya existen en 2021 Q1, se omite descarga.
✅ Archivos ya existen en 2021 Q2, se omite descarga.
✅ Archivos ya existen en 2021 Q3, se omite descarga.
✅ Archivos ya existen en 2021 Q4, se omite descarga.
✅ Archivos ya existen en 2022 Q1, se omite descarga.
✅ Archivos ya exis

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

⏭️ 2019-01-01_performance_fixed_tiles.parquet ya estaba cargado en la base. Se omite inserción.

🔄 Procesando /content/gdrive/MyDrive/trabajoGrado/ookla_colombia/2019/quarter=2/2019-04-01_performance_fixed_tiles.parquet...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

⏭️ 2019-04-01_performance_fixed_tiles.parquet ya estaba cargado en la base. Se omite inserción.

🔄 Procesando /content/gdrive/MyDrive/trabajoGrado/ookla_colombia/2019/quarter=3/2019-07-01_performance_fixed_tiles.parquet...


...

⏭️ 2024-04-01_performance_fixed_tiles.parquet ya estaba cargado en la base. Se omite inserción.

🔄 Procesando /content/gdrive/MyDrive/trabajoGrado/ookla_colombia/2024/quarter=3/2024-07-01_performance_fixed_tiles.parquet...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

⏭️ 2024-07-01_performance_fixed_tiles.parquet ya estaba cargado en la base. Se omite inserción.

🔄 Procesando /content/gdrive/MyDrive/trabajoGrado/ookla_colombia/2024/quarter=4/2024-10-01_performance_fixed_tiles.parquet...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

⏭️ 2024-10-01_performance_fixed_tiles.parquet ya estaba cargado en la base. Se omite inserción.

🧾 Log actualizado con 24 archivos procesados.
📊 Total insertados en esta ejecución: 0 registros.
🎯 Base Ookla lista para análisis (Notebook 3 - INCTIC).

🎯 Proceso completado. Base Ookla lista para análisis (Notebook 3 - INCTIC).


In [6]:
# ============================================================
# === 9. Revisión de la base DuckDB construida ===============
# ============================================================

import duckdb
import pandas as pd

# Ruta a la base generada
db_path = "/content/gdrive/MyDrive/trabajoGrado/ookla_datos/ookla_colombia.duckdb"

# Conexión a la base
con = duckdb.connect(db_path)

print("📂 Tablas disponibles en la base:")
tablas = con.execute("SHOW TABLES").df()
display(tablas)

# ------------------------------------------------------------
# Mostrar información general de la tabla principal
# ------------------------------------------------------------
print("\n📊 Estructura de la tabla 'ookla_geo':")
estructura = con.execute("PRAGMA table_info('ookla_geo')").df()
display(estructura)

# ------------------------------------------------------------
# Conteo total de registros
# ------------------------------------------------------------
total_registros = con.execute("SELECT COUNT(*) AS total FROM ookla_geo").fetchone()[0]
print(f"\n🧮 Total de registros en la base: {total_registros:,}")

# ------------------------------------------------------------
# Número de variables
# ------------------------------------------------------------
num_columnas = len(estructura)
print(f"🔢 Total de variables (columnas): {num_columnas}")

# ------------------------------------------------------------
# Vista previa de los primeros registros
# ------------------------------------------------------------
print("\n👀 Vista previa de la base (10 registros):")
vista = con.execute("SELECT * FROM ookla_geo LIMIT 10").df()
display(vista)

# ------------------------------------------------------------
# Estadísticas básicas de las principales métricas
# ------------------------------------------------------------
print("\n📈 Estadísticas descriptivas de variables numéricas:")
stats = con.execute("""
    SELECT
        AVG(CAST(avg_d_kbps AS DOUBLE)) AS promedio_descarga_kbps,
        AVG(CAST(avg_u_kbps AS DOUBLE)) AS promedio_subida_kbps,
        AVG(CAST(avg_lat_ms AS DOUBLE)) AS promedio_latencia_ms,
        COUNT(*) AS total_registros
    FROM ookla_geo
""").df()
display(stats)

con.close()


📂 Tablas disponibles en la base:


,name
0,ookla_geo



📊 Estructura de la tabla 'ookla_geo':


,cid,name,type,notnull,dflt_value,pk
0,0,quadkey,VARCHAR,False,None,False
1,1,tile,VARCHAR,False,None,False
2,2,avg_d_kbps,BIGINT,False,None,False
3,3,avg_u_kbps,BIGINT,False,None,False
4,4,avg_lat_ms,BIGINT,False,None,False
5,5,tests,BIGINT,False,None,False
6,6,devices,BIGINT,False,None,False
7,7,lat,DOUBLE,False,None,False
8,8,lon,DOUBLE,False,None,False
9,9,shapeName_left,VARCHAR,False,None,False



🧮 Total de registros en la base: 133,473
🔢 Total de variables (columnas): 26

👀 Vista previa de la base (10 registros):


,quadkey,tile,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,lat,lon,shapeName_left,...,shapeISO__dep,shapeID__dep,shapeGroup__dep,shapeType__dep,municipio,departamento,tile_x,tile_y,avg_lat_down_ms,avg_lat_up_ms
0,0322123322313320,"POLYGON((-74.20166015625 11.2753866926, -74.19...",719,8110,22,1,1,11.272693,-74.198914,None,...,None,None,None,None,Santa Marta,Magdalena,None,None,None,None
1,0322123322313323,"POLYGON((-74.1961669921875 11.2699995030222, -...",6302,1601,64,11,10,11.267306,-74.193420,None,...,None,None,None,None,None,None,None,None,None,None
2,0322123322313330,"POLYGON((-74.190673828125 11.2753866926, -74.1...",10612,782,51,1,1,11.272693,-74.187927,None,...,None,None,None,None,Santa Marta,Magdalena,None,None,None,None
3,0322123322313332,"POLYGON((-74.190673828125 11.2699995030222, -7...",3796,1007,71,42,10,11.267306,-74.187927,None,...,None,None,None,None,Santa Marta,Magdalena,None,None,None,None
4,0322123322331023,"POLYGON((-74.2181396484375 11.2484497357683, -...",7817,6163,85,25,18,11.245756,-74.215393,None,...,None,None,None,None,None,None,None,None,None,None
5,0322123322331030,"POLYGON((-74.212646484375 11.2538373288317, -7...",10364,2150,13,2,1,11.251144,-74.209900,None,...,None,None,None,None,Santa Marta,Magdalena,None,None,None,None
6,0322123322331031,"POLYGON((-74.2071533203125 11.2538373288317, -...",8387,1941,170,4,1,11.251144,-74.204407,None,...,None,None,None,None,Santa Marta,Magdalena,None,None,None,None
7,0322123322331032,"POLYGON((-74.212646484375 11.2484497357683, -7...",18129,15233,63,44,34,11.245756,-74.209900,None,...,None,None,None,None,Santa Marta,Magdalena,None,None,None,None
8,0322123322331033,"POLYGON((-74.2071533203125 11.2484497357683, -...",14176,3706,38,30,4,11.245756,-74.204407,None,...,None,None,None,None,Santa Marta,Magdalena,None,None,None,None
9,0322123322331101,"POLYGON((-74.1961669921875 11.2646122125044, -...",6445,797,60,2,2,11.261919,-74.193420,None,...,None,None,None,None,Santa Marta,Magdalena,None,None,None,None



📈 Estadísticas descriptivas de variables numéricas:


,promedio_descarga_kbps,promedio_subida_kbps,promedio_latencia_ms,total_registros
0,12860.163411,6911.264705,76.903126,133473


In [7]:
import os

db_path = "/content/gdrive/MyDrive/trabajoGrado/ookla_datos/ookla_colombia.duckdb"
size_mb = os.path.getsize(db_path) / (1024*1024)
print(f"📦 Tamaño actual del archivo DuckDB: {size_mb:.2f} MB")

# Comprobar integridad: abrir y contar registros
import duckdb
con = duckdb.connect(db_path)
total = con.execute("SELECT COUNT(*) FROM ookla_geo").fetchone()[0]
print(f"✅ Total de registros verificados dentro de la base: {total:,}")
con.close()


📦 Tamaño actual del archivo DuckDB: 23.76 MB
✅ Total de registros verificados dentro de la base: 133,473
